In [5]:
import geopandas as gpd
import numpy as np
import folium
from scipy.spatial.distance import cdist


In [6]:

# 读取GeoJSON文件
geojson_path = './combined_coordinates.geojson'
gdf = gpd.read_file(geojson_path)

# 提取坐标点
coordinates = gdf.geometry.apply(lambda geom: (geom.x, geom.y)).tolist()

# 计算距离矩阵
def haversine(coord1, coord2):
    lat1, lon1 = np.radians(coord1)
    lat2, lon2 = np.radians(coord2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of earth in kilometers
    return c * r

distance_matrix = cdist(coordinates, coordinates, metric=lambda u, v: haversine(u, v))

# 创建地图
m = folium.Map(location=[np.mean([coord[1] for coord in coordinates]), np.mean([coord[0] for coord in coordinates])], zoom_start=12)

# 添加点和距离线
for i, coord1 in enumerate(coordinates):
    for j, coord2 in enumerate(coordinates):
        if i < j:  # 只计算一次每对距离
            distance = distance_matrix[i, j]
            folium.PolyLine([coord1, coord2], color="blue", weight=1.5, opacity=0.7, tooltip=f'{distance:.2f} km').add_to(m)
    folium.Marker(location=list(coord1), popup=f'Point {i+1}').add_to(m)

# 保存地图为HTML文件
map_path = './distance_map.html'
m.save(map_path)
map_path


'./distance_map.html'

In [7]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

# Define the URL of the HTML file
html_file = './distance_map.html'

# Define the directory to save the downloaded files
save_directory = './resources'

# Create the save directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

# Read the HTML file
with open(html_file, 'r') as f:
    html_content = f.read()

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find all the <a> tags with href attribute
links = soup.find_all('a', href=True)

# Iterate over the links
for link in links:
    # Get the URL of the linked file
    file_url = link['href']
    
    # Parse the URL to get the filename
    parsed_url = urlparse(file_url)
    filename = os.path.basename(parsed_url.path)
    
    # Download the file
    response = requests.get(file_url)
    
    # Save the file to the specified directory
    save_path = os.path.join(save_directory, filename)
    with open(save_path, 'wb') as f:
        f.write(response.content)
    
    # Change the link to the local file
    local_link = os.path.join('resources', filename)
    link['href'] = local_link

# Save the modified HTML file
modified_html_file = os.path.join(save_directory, 'modified_distance_map.html')
with open(modified_html_file, 'w') as f:
    f.write(str(soup))